# Manually Tracking instructions:
The following code allows us to analyze the time-series skyrmion data to obtain good seeds for gradient descent. The skyrmion data can be obtained from the dropbox in a `<start_numor>.npz` format.

## How it works:
1) Open and run `view_w_slider.ipynb` to get a qualitative and interactive viewing of the data. This will help your intuition in clicking through the frames. Leave this window running for the duration of analysis for assistance.
2) Navigate back to this file
3) Load data and other functions `1.0`
4) Analyze and click through data `1.1`
5) Make gif for validation `1.2`
6) When satisfied with the results, save offset1 as an npz `1.3`
7) Now, follow a similar protocol to find offset2 `2.0`, `2.1`, and `2.2`

LMK if you have questions or if you run into problems. Feel free to change hyperparameters however it may help you!

## *In the world of skyrmions, every click is a step towards greatness. May your mouse be swift and your offsets precise!🔥*

In [ ]:
import matplotlib
matplotlib.use('TkAgg')  # Use TkAgg backend for external window
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sys
import os
from scipy.ndimage import gaussian_filter
import tempfile
import torch
import pickle
import tempfile
import subprocess
import os


## 1.0: Load data and necessary functions

In [ ]:
# Example usage:
numor = 54863
file_path = f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/experimental_data/{numor}.npz'
data = np.load(file_path)['data']
image_stack = data
domain = 1
# Run the plotting script and retrieve recorded angles

h, w = image_stack[0].shape[:2]
cx, cy = w // 2, h // 2
# Create grid with coordinates relative to the center
x_grid, y_grid = np.meshgrid(np.arange(w), np.arange(h))
DATA_THETA = np.arctan2(y_grid - cy, x_grid - cx)

n_folds = 6
k = 8

def mask_and_blur_images(array, sigma=1):
    '''Masks signal inside radius of 14 and outside radius of 30 and applies gaussian blur.'''
    x, y = np.meshgrid(np.arange(128), np.arange(128))
    radius = np.sqrt((x - 64) ** 2 + (y - 62) ** 2)
    mask1 = radius <= 14
    mask2 = radius >= 30
    masked_data = array.copy()
    masked_data[mask1] = 0
    masked_data2 = masked_data.copy()
    masked_data2[mask2] = 0
    blurred_data = gaussian_filter(masked_data2, sigma=sigma)
    return blurred_data

def filter_function(k, theta, n_folds=n_folds):
    # Compute the filter using the provided angular grid.
    filt = np.exp(k * np.log((np.cos(n_folds / 2 * theta))**2))
    return filt

def normalize_min_max(data):
    try:
        array = data.copy()
    except AttributeError:
        array = np.array(data, copy=True)
    # Check for empty arrays
    if array.size == 0:
        raise ValueError("Cannot normalize an empty array.")
    # Compute the minimum and maximum values.
    array_min = np.min(array)
    array_max = np.max(array)
    # If the data is constant, avoid division by zero.
    if array_max == array_min:
        return np.zeros_like(array)
    # Perform the normalization.
    norm_array = (array - array_min) / (array_max - array_min)
    return norm_array


## 1.1: Calculate offset 1
click where the bragg peak is. Make sure to track the same bragg peak through series.

In [ ]:

def run_plotting_script(image_stack, numor, domain):
    """
    Serializes the image_stack, runs the plotting script, and retrieves the recorded angles.
    
    :param image_stack: A 3D (N,H,W) or 4D (N,H,W,C) numpy array of images.
    :return: A list of recorded azimuthal angles.
    """
    # Create temporary files for input and output
    with tempfile.NamedTemporaryFile(delete=False, suffix='.pkl') as input_file, \
         tempfile.NamedTemporaryFile(delete=False, suffix='.pkl') as output_file:
        
        input_path = input_file.name
        output_path = output_file.name

        # Serialize the image stack to the input file
        pickle.dump(image_stack, input_file)
        input_file.flush()

    try:
        # Path to the plotting script
        plotter_script = 'record_angles_plotter.py'  # Ensure this script is in the current directory

        # Check if the plotting script exists
        if not os.path.exists(plotter_script):
            raise FileNotFoundError(f"Plotting script '{plotter_script}' not found.")

        # Run the plotting script as a subprocess
        print("Launching the interactive plot window...")
        subprocess.run(['python', plotter_script, input_path, output_path, str(numor), str(domain)], check=True)
        print("Interactive plot window closed.")

        # After the subprocess finishes, deserialize the recorded angles
        with open(output_path, 'rb') as f:
            recorded_angles = pickle.load(f)

        return recorded_angles

    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running the plotting script: {e}")
        return []
    finally:
        # Clean up temporary files
        os.remove(input_path)
        os.remove(output_path)

'''
Easy: 69958
Medium: 56059
Hard: 46349
'''

offset1 = run_plotting_script(image_stack, numor, domain)
print("Recorded azimuthal angles:", offset1)



In [ ]:
%matplotlib inline

# plot your hand calculated data
plt.plot(offset1, label='offset1')
plt.title('offset 1')
plt.ylabel('Angle (degrees)')
plt.xlabel('Image index')
plt.legend()
plt.show()

In [ ]:
# plot your hand calculated data
plt.plot(offset1-offset1[0], label='offset1')
plt.title('offset 1')
plt.ylabel('Angle (degrees)')
plt.xlabel('Image index')
plt.legend()
plt.show()

## 1.2: Save angles as npz
Change file name to `<start-numor>_offset1_seed.npz`.

In [ ]:
file_name = f'cm_{numor}_offset1_seed.npz'

if os.path.exists(file_name):
    raise FileExistsError(f"Warning: The file '{file_name}' already exists! Be careful not to overwrite data.")
else:
    np.savez(file_name, data=(np.array(offset1)-30))
    print(f"File saved as '{file_name}'")

## 1.3: Gif creation
Check your work with this

In [ ]:
from matplotlib.animation import FuncAnimation
from PIL import Image

data = np.load(file_path)['data']
# Create a list to hold the individual frames
frames = []

for i in range(len(offset1)):
    fig, ax = plt.subplots()
    im_norm = normalize_min_max(mask_and_blur_images(data[i]))

    filt = filter_function(8, DATA_THETA - np.deg2rad(offset1[i]), n_folds=6) # change n_folds and k for your viewing pleasure
    cax = ax.imshow(im_norm + filt, cmap='viridis')
    ax.set_title(f'index: {i}')
    plt.axis('off')  # Hide axes
    
    # Convert the plot to a PIL image
    fig.canvas.draw()
    image = np.array(fig.canvas.renderer.buffer_rgba())
    pil_image = Image.fromarray(image)
    frames.append(pil_image)
    plt.close(fig)

# Save the frames as a GIF
output_gif_path = f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/{numor}_domain1.gif' # Change the name to '<start_numor>_hand_calc.gif'
if os.path.exists(output_gif_path):
    raise FileExistsError(f"Warning: The file '{output_gif_path}' already exists! Be careful not to overwrite data.")
else:
    frames[0].save(output_gif_path, save_all=True, append_images=frames[1:], loop=0, duration=200)
    print(f"File saved as '{output_gif_path}'")

## 2.0: Calculate offset 2

In [ ]:
# Run the plotting script and retrieve recorded angles for second domain
matplotlib.use('TkAgg')  # Use TkAgg backend for external window
domain = 2
offset2 = run_plotting_script(image_stack, numor, domain)
print("Recorded azimuthal angles:", offset2)

## 2.1: Save angles as npz
Change file name to `<start-numor>_offset2_seed.npz`.

In [ ]:
file_name = f'cm_{numor}_offset2_seed.npz'

if os.path.exists(file_name):
    raise FileExistsError(f"Warning: The file '{file_name}' already exists! Be careful not to overwrite data.")
else:
    np.savez(file_name, data=offset2)
    print(f"File saved as '{file_name}'")

## 2.2: Make final gif

In [ ]:
from matplotlib.animation import FuncAnimation
from PIL import Image
%matplotlib inline
# Create a list to hold the individual frames
frames = []
offset2 = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_{numor}_offset2_seed.npz')['data']
offset1 = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_{numor}_offset1_seed.npz')['data']
for i in range(data.shape[0]):
    fig, ax = plt.subplots()
    im_norm = normalize_min_max(mask_and_blur_images(data[i]))

    filt = filter_function(8, DATA_THETA - np.deg2rad(offset1[i]), n_folds=6)
    filt2 = 2*filter_function(8, DATA_THETA - np.deg2rad(offset2[i]+30), n_folds=6) # change n_folds and k for your viewing pleasure
    cax = ax.imshow(im_norm + filt + filt2, cmap='viridis')
    ax.set_title(f'index: {i}')
    plt.axis('off')  # Hide axes
    
    # Convert the plot to a PIL image
    fig.canvas.draw()
    image = np.array(fig.canvas.renderer.buffer_rgba())
    pil_image = Image.fromarray(image)
    frames.append(pil_image)
    plt.close(fig)

# Save the frames as a GIF
final_gif_file = f'cm_{numor}_seeds.gif' # Change the name to '<start_numor>_seeds.gif'

if os.path.exists(final_gif_file):
    raise FileExistsError(f"Warning: The file '{final_gif_file}' already exists! Be careful not to overwrite data.")
else:
    frames[0].save(final_gif_file, save_all=True, append_images=frames[1:], loop=0, duration=200)
    print(f"File saved as '{final_gif_file}'")


## adjustment function

In [ ]:
def adjust_offset_ratchet_model(offset_list, tolerance_angle, jump_angle):
    '''
    jump_angle: either 30 or 60, controls the adjustment value
    tolerance_angle: tolerance we give GD to move in opposite direction
    '''
    angle_below_offset = jump_angle - tolerance_angle
    adjusted_offsets = []
    prev_offset = offset_list[0]
    for index, offset in enumerate(offset_list):
        # keep invalid frames
        if np.isnan(offset):
            adjusted_offsets.append(offset)
            continue
        if index == 0:
            adjusted_offsets.append(offset)
            prev_offset = offset
        else:
            offset_range = (prev_offset - angle_below_offset, prev_offset + tolerance_angle)
            while not (offset_range[0] <= offset <= offset_range[1]):
                offset += jump_angle if offset < offset_range[0] else -jump_angle
            adjusted_offsets.append(offset)
            prev_offset = offset
    return np.array(adjusted_offsets)

In [ ]:
# plot your hand calculated data
%matplotlib inline
offset1 = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_{numor}_offset1_seed.npz')['data']
offset2 = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_{numor}_offset2_seed.npz')['data']
offset1oob = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/{numor}_offsets.npz')['offset1']
offset2oob = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/{numor}_offsets.npz')['offset2']
adjusted_offset1 = adjust_offset_ratchet_model(offset1oob, 20, 30)
adjusted_offset2 = adjust_offset_ratchet_model(offset2oob, 15, 30)

# Get the viridis colormap
viridis = plt.cm.viridis

# Generate a list of colors by indexing
colors = viridis(np.linspace(0, 1, 10))


plt.plot(offset1-offset1[0], label='offset1 hand calc', color='red') # hand calc
plt.plot(offset2-offset2[0], label='offset2 hand calc', linestyle='--', color='red') # hand calc
plt.plot(adjusted_offset1-adjusted_offset1[0], label='offset1 post-process ratchet', color='green') # out-of-box ratchet
plt.plot(adjusted_offset2-adjusted_offset2[0],  label='offset2 post-process ratchet', linestyle='--', color='green') # out-of-box ratchet
plt.plot(offset1oob-offset1oob[0], label='offset1 out-of-box', color='blue') # out-of-box
plt.plot(offset2oob-offset2oob[0], label='offset2 out-of-box', linestyle='--', color='blue') # out-of-box

plt.title('offset 1 and 2')
plt.ylabel('Angle (degrees)')
plt.xlabel('Image index')
plt.legend()
plt.grid()
plt.show()

## Congrats! You're done! :-)
Please upload offset1, offset2, and final gif to the Dropbox when finished!

## 69958: Easy

In [ ]:
# plot your hand calculated data
offset1 = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_69958_offset2_seed.npz')['data']
offset2 = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_69958_offset1_seed.npz')['data']
offset1oob = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/69958_offsets.npz')['offset1']
offset2oob = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/69958_offsets.npz')['offset2']
tolerance1 = 5
tolerance2 = 15
adjusted_offset1 = adjust_offset_ratchet_model(offset1oob, tolerance1, 30)
adjusted_offset2 = adjust_offset_ratchet_model(offset2oob, tolerance2, 30)

# Offset 1
plt.plot(offset1 - offset1[0], label='offset1 hand calc', color='red', linestyle='solid')  # Hand calc
plt.plot(adjusted_offset1 - adjusted_offset1[0], label=f'offset1 post-process ratchet, tolerance={tolerance1}', color='red', linestyle='dashed', alpha=0.5)  # Ratchet
plt.plot(offset1oob - offset1oob[0], label='offset1 out-of-box', color='red', linestyle='dotted', alpha=0.5)  # Out-of-box

# Offset 2
plt.plot(offset2 - offset2[0], label='offset2 hand calc', color='blue', linestyle='solid')  # Hand calc
plt.plot(adjusted_offset2 - adjusted_offset2[0], label=f'offset2 post-process ratchet, tolerance={tolerance2}', color='blue', linestyle='dashed', alpha=0.5)  # Ratchet
plt.plot(offset2oob - offset2oob[0], label='offset2 out-of-box', color='blue', linestyle='dotted', alpha=0.5)  # Out-of-box




plt.title('offset 1 and 2 69958')
plt.ylabel('Angle (degrees)')
plt.xlabel('Image index')
plt.legend()
plt.grid()
plt.show()

## 56059: Medium

In [ ]:
# plot your hand calculated data
offset2 = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_56059_offset2_seed.npz')['data']
offset1 = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_56059_offset1_seed.npz')['data']
offset1oob = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/56059_offsets.npz')['offset1']
offset2oob = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/56059_offsets.npz')['offset2']
tolerance1 = 5
tolerance2 = 8
adjusted_offset1 = adjust_offset_ratchet_model(offset1oob, tolerance1, 30)
adjusted_offset2 = adjust_offset_ratchet_model(offset2oob, tolerance2, 30)

# Offset 1
plt.plot(offset1 - offset1[0], label='offset1 hand calc', color='red', linestyle='solid')  # Hand calc
plt.plot(adjusted_offset1 - adjusted_offset1[0], label=f'offset1 post-process ratchet, tolerance={tolerance1}', color='red', linestyle='dashed', alpha=0.5)  # Ratchet
plt.plot(offset1oob - offset1oob[0], label='offset1 out-of-box', color='red', linestyle='dotted', alpha=0.5)  # Out-of-box

# Offset 2
plt.plot(offset2 - offset2[0], label='offset2 hand calc', color='blue', linestyle='solid')  # Hand calc
plt.plot(adjusted_offset2 - adjusted_offset2[0], label=f'offset2 post-process ratchet, tolerance={tolerance2}', color='blue', linestyle='dashed', alpha=0.5)  # Ratchet
plt.plot(offset2oob - offset2oob[0], label='offset2 out-of-box', color='blue', linestyle='dotted', alpha=0.5)  # Out-of-box




plt.title('offset 1 and 2 56059')
plt.ylabel('Angle (degrees)')
plt.xlabel('Image index')

plt.legend()
plt.grid()
plt.show()

## 46349: hard

In [ ]:
offset2 = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/cm_46349_offset1_seed.npz')['data']
offset1 = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/cm_46349_offset2_seed.npz')['data']
offset1oob = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/46349_offsets.npz')['offset1']
offset2oob = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/46349_offsets.npz')['offset2']
tolerance1 = 20
tolerance2 = 20
adjusted_offset1 = adjust_offset_ratchet_model(offset1oob, tolerance1, 30)
adjusted_offset2 = adjust_offset_ratchet_model(offset2oob, tolerance2, 30)

# Offset 1
plt.plot(offset1 - offset1[0], label='offset1 hand calc', color='red', linestyle='solid')  # Hand calc
plt.plot(adjusted_offset1 - adjusted_offset1[0], label=f'offset1 post-process ratchet, tolerance={tolerance1}', color='red', linestyle='dashed', alpha=0.5)  # Ratchet
plt.plot(offset1oob - offset1oob[0], label='offset1 out-of-box', color='red', linestyle='dotted', alpha=0.5)  # Out-of-box

# Offset 2
plt.plot(offset2 - offset2[0], label='offset2 hand calc', color='blue', linestyle='solid')  # Hand calc
plt.plot(adjusted_offset2 - adjusted_offset2[0], label=f'offset2 post-process ratchet, tolerance={tolerance2}', color='blue', linestyle='dashed', alpha=0.5)  # Ratchet
plt.plot(offset2oob - offset2oob[0], label='offset2 out-of-box', color='blue', linestyle='dotted', alpha=0.5)  # Out-of-box



plt.title('offset 1 and 2 46349')
plt.ylabel('Angle (degrees)')
plt.xlabel('Image index')

plt.legend()
plt.grid()
plt.show()

## Compare hand-calc

In [ ]:


# Hard difficulty
offset1cmhard = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_46349_offset2_seed.npz')['data']
offset2cmhard = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_46349_offset1_seed.npz')['data']
offset1ychard = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/yc_46349_offset2_seed.npz')['data']
offset2ychard = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/yc_46349_offset1_seed.npz')['data']
offset1drhard = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/dr_46349_offset2_seed.npz')['data']
offset2drhard = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/dr_46349_offset1_seed.npz')['data']
offset1nchard = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/nc_46349_offset2_seed.npz')['data']
offset2nchard = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/nc_46349_offset1_seed.npz')['data']
offset1oob4 = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/46349_offsets.npz')['offset1']
offset2oob4 = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/46349_offsets.npz')['offset2']

# Medium difficulty
offset1cmmed = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_56059_offset1_seed.npz')['data']
offset2cmmed = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_56059_offset2_seed.npz')['data']
offset1ycmed = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/yc_56059_offset1_seed.npz')['data']
offset2ycmed = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/yc_56059_offset2_seed.npz')['data']
offset1drmed = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/dr_56059_offset1_seed.npz')['data']
offset2drmed = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/dr_56059_offset2_seed.npz')['data']
offset1ncmed = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/nc_56059_offset1_seed.npz')['data']
offset2ncmed = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/nc_56059_offset2_seed.npz')['data']
offset1oob5 = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/56059_offsets.npz')['offset1']
offset2oob5 = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/56059_offsets.npz')['offset2']

# Easy difficulty
offset1cmeasy = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_69958_offset2_seed.npz')['data']
offset2cmeasy = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/cm_69958_offset1_seed.npz')['data']
offset1yceasy = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/yc_69958_offset2_seed.npz')['data']
offset2yceasy = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/yc_69958_offset1_seed.npz')['data']
offset1dreasy = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/dr_69958_offset2_seed.npz')['data']
offset2dreasy = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/dr_69958_offset1_seed.npz')['data']
offset1nceasy = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/nc_69958_offset2_seed.npz')['data']
offset2nceasy = np.load('/Users/cadenmyers/billingelab/dev/sym_adapted_filts/working_code/seed_generating/seed_data/nc_69958_offset1_seed.npz')['data']
offset1oob4 = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/69958_offsets.npz')['offset1']
offset2oob4 = np.load(f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/analysis-generated_data/69958_offsets.npz')['offset2']

# Define colors for plotting
colors = {
    "cm": "blue",
    "yc": "red",
    "dr": "green",
    "nc": "purple",
    "oob": 'orange'
}

# Hard difficulty
plt.figure(figsize=(10, 10))
plt.plot(offset1oob4-offset1oob4[0], color=colors['oob'], label=f'offset1 oob hard', linestyle='dotted')
plt.plot(offset2oob4-offset2oob4[0], color=colors['oob'], label=f'offset2 oob hard', linestyle='-')
plt.plot(offset1cmhard-offset1cmhard[0], label=f'offset1 cm hard', color=colors['cm'], linestyle='dotted')
plt.plot(offset2cmhard-offset2cmhard[0], label=f'offset2 cm hard', color=colors['cm'], linestyle='-')
plt.plot(offset1ychard-offset1ychard[0], label=f'offset1 yc hard', color=colors["yc"], linestyle='dotted')
plt.plot(offset2ychard-offset2ychard[0], label=f'offset2 yc hard', color=colors["yc"], linestyle='-')
plt.plot(offset1drhard-offset1drhard[0], label=f'offset1 dr hard', color=colors["dr"], linestyle='dotted')
plt.plot(offset2drhard-offset2drhard[0], label=f'offset2 dr hard', color=colors["dr"], linestyle='-')
plt.plot(offset1nchard-offset1nchard[0], label=f'offset1 nc hard', color=colors["nc"], linestyle='dotted')
plt.plot(offset2nchard-offset2nchard[0], label=f'offset2 nc hard', color=colors["nc"], linestyle='-')
plt.title("46349")
plt.grid(True)
plt.legend()
plt.show()

# Medium difficulty
plt.figure(figsize=(10, 10))
plt.plot(offset1oob5-offset1oob5[0], color=colors['oob'], label=f'offset1 oob medium', linestyle='dotted')
plt.plot(offset2oob5-offset2oob5[0], color=colors['oob'], label=f'offset2 oob medium', linestyle='-')
plt.plot(offset1cmmed-offset1cmmed[0], label=f'offset1 cm medium', color=colors['cm'], linestyle='dotted')
plt.plot(offset2cmmed-offset2cmmed[0], label=f'offset2 cm medium', color=colors['cm'], linestyle='-')
plt.plot(offset1ycmed-offset1ycmed[0], label=f'offset1 yc medium', color=colors["yc"], linestyle='dotted')
plt.plot(offset2ycmed-offset2ycmed[0], label=f'offset2 yc medium', color=colors["yc"], linestyle='-')
plt.plot(offset1drmed-offset1drmed[0], label=f'offset1 dr medium', color=colors["dr"], linestyle='dotted')
plt.plot(offset2drmed-offset2drmed[0], label=f'offset2 dr medium', color=colors["dr"], linestyle='-')
plt.plot(offset1ncmed-offset1ncmed[0], label=f'offset1 nc medium', color=colors["nc"], linestyle='dotted')
plt.plot(offset2ncmed-offset2ncmed[0], label=f'offset2 nc medium', color=colors["nc"], linestyle='-')
plt.title("56059")
plt.grid(True)
plt.legend()
plt.show()

# Easy difficulty
plt.figure(figsize=(10, 10))
plt.plot(offset1oob4-offset1oob4[0], color=colors['oob'], label=f'offset1 oob easy', linestyle='dotted')
plt.plot(offset2oob4-offset2oob4[0], color=colors['oob'], label=f'offset2 oob easy', linestyle='-')
plt.plot(offset1cmeasy-offset1cmeasy[0], label=f'offset1 cm easy', color=colors['cm'], linestyle='dotted')
plt.plot(offset2cmeasy-offset2cmeasy[0], label=f'offset2 cm easy', color=colors['cm'], linestyle='-')
plt.plot(offset1yceasy-offset1yceasy[0], label=f'offset1 yc easy', color=colors["yc"], linestyle='dotted')
plt.plot(offset2yceasy-offset2yceasy[0], label=f'offset2 yc easy', color=colors["yc"], linestyle='-')
plt.plot(offset1dreasy-offset1dreasy[0], label=f'offset1 dr easy', color=colors["dr"], linestyle='dotted')
plt.plot(offset2dreasy-offset2dreasy[0], label=f'offset2 dr easy', color=colors["dr"], linestyle='-')
plt.plot(offset1nceasy-offset1nceasy[0], label=f'offset1 nc easy', color=colors["nc"], linestyle='dotted')
plt.plot(offset2nceasy-offset2nceasy[0], label=f'offset2 nc easy', color=colors["nc"], linestyle='-')
plt.title("69958")
plt.legend()
plt.grid(True)
plt.show()
